# Data-Based Forest Management with Uncertainties and Multiple Objectives

## We introduce more uncertainty to our data so that we can model the real difficulties in decision making

### Define the shufling of the datasets

In [ ]:
no_Stands = 300
no_Dataset = 25
no_realization = 50

In [ ]:
#Uncomment to make a new AB file

#from random import shuffle, randint

#AB = [[randint(1,no_Dataset) for j in range(no_realization)] for i in range(no_Stands)]

#AB_str = "AB = ["
#for part in AB:
#    AB_str+=str(part)
#AB_str+="];"
#print_file = open("AB.dat","w")
#print_file.write(AB_str)
#print_file.close()


## Defining the details of the problem in a dictionary

In [ ]:
problem = {
    'data': """
int no_Stands = ...;
int no_Dataset = ...;    
int no_Variables = ...; //Actually outcomes: 1 - 6 incomes, 7-12 Biodiversity, last 1 (don't know why)
int max_no_Schedules = ...;
int no_realization = """+str(no_realization)+""";
range Schedules = 1..max_no_Schedules;
range Datasets = 1..no_Dataset;
range Stands = 1..no_Stands;
range Realizations = 1..no_realization;
int AB[Stands][Realizations]=...;
//The data was organized in a slightly different fashion.
float Data_MKA[1..no_Stands][1..max_no_Schedules][1..no_Dataset][1..no_Variables+1] = ...;
""",
    'variables': {
        0:{
            'name': "chosen_schedule",
            'opl': "dvar float chosen_schedule[1..no_Stands][1..max_no_Schedules] in 0..1;"
        },
        1:{
            'name': "realization_chooser_income",
            'opl': "dvar int+ realization_chooser_income[Realizations] in 0..1; //dummy for choosing the datasets considered"
        },
        2:{
            'name': "realization_chooser_biodiversity",
            'opl': "dvar int+ realization_chooser_biodiversity[Realizations] in 0..1; //dummy for choosing the datasets considered"
        },
    3:{
            'name': "quantile_income",
            'opl': "dvar float+ quantile_income; //dummy for choosing the datasets considered"
        },
    4:{
            'name': "quantile_biodiversity",
            'opl': "dvar float+ quantile_biodiversity; //dummy for choosing the datasets considered"
        }
    },
    'objectives': {
        0:{
            'name': "Minimum_income_in_quantile",
            'minimized': "-1",
            'opl':"""
min(i in 1..6, realization in Realizations) 
(
sum(schedule in 1..max_no_Schedules, stand in 1..no_Stands)
(
chosen_schedule[stand][schedule]*Data_MKA[stand][schedule][AB[stand][realization]][i] //If realization is in the chosen ones, then the income is given by data
+ (1-realization_chooser_income[realization])*max(realization2 in Realizations)Data_MKA[stand][schedule][AB[stand][realization2]][i] //If realization is not in the chosen ones, then the income is 100000.. (a big number)
)
)
""",
        'comment': """
//Minimum income in the quantile
        """},
        1:{
    'name': "Minimum_biodiversity_in_quantile",
            'minimized': "-1",
            'opl':"""
min(j in 7..12, realization in Realizations) 
(
sum(stand in 1..no_Stands, schedule in 1..max_no_Schedules)
(chosen_schedule[stand][schedule]*Data_MKA[stand][schedule][AB[stand][realization]][j]
+ (1-realization_chooser_biodiversity[realization])*max(realization2 in Realizations)Data_MKA[stand][schedule][AB[stand][realization2]][j] //If realization is not in the chosen ones, then the biodiversity is 100000.. (a big number)
)
)
""",
        'comment': """
//minimum biodiversity in the quantile
        """
        },
        2:{
    'name': "Expected_minimum_income",
            'minimized': "-1",
            'opl':"""
min(i in 1..6)
(
sum(realization in Realizations) 
(
sum(schedule in 1..max_no_Schedules, stand in 1..no_Stands)
chosen_schedule[stand][schedule]*Data_MKA[stand][schedule][AB[stand][realization]][i]
)/no_realization
)
""",
        'comment': """
//minimum expected income
        """},
        3:{
    'name': "Expected_minimum_biodiversity",
            'minimized': "-1",
            'opl':"""
min(j in 7..12)
(
sum(realization in Realizations) 
(
sum(stand in 1..no_Stands, schedule in 1..max_no_Schedules)
chosen_schedule[stand][schedule]*Data_MKA[stand][schedule][AB[stand][realization]][j]
)/no_realization
)
""",
        'comment': """
//Minimum expected biodiversity
        """},
        4:{
    'name': "Reliability_income",
            'minimized': "-1",
            'opl':"""
 sum(realization in Realizations) realization_chooser_income[realization]/no_realization
""",
        'comment': """
//The probability of being in the studied quantile
        """
	},
        5:{
    'name': "Reliability_biodiversity",
            'minimized': "-1",
            'opl':"""
 sum(realization in Realizations) realization_chooser_biodiversity[realization]/no_realization;
""",
        'comment': """
//The probability of being in the studied quantile
        """
	}
    },
    'constraints':{
        0:{
        'opl':"""
forall(stand in 1..no_Stands)
sum(schedule in 1..max_no_Schedules) chosen_schedule[stand][schedule] == 1;
""",
        'comment':"""//Only one schedule can be chosen for each stand"""
                            },
	1:{
	    'opl':"""
sum(realization in Realizations) realization_chooser_income[realization]/no_realization >= quantile_income;
	    """,
	    'comment':"""
//quantile of the realizations must be chosen
"""},
    2:{
	    'opl':"""
sum(realization in Realizations) realization_chooser_biodiversity[realization]/no_realization >= quantile_biodiversity;
	    """,
	    'comment':"""
//quantile of the realizations must be chosen
"""},
	3:{
	    'opl':"""
 quantile_income>=1/"""+str(no_realization)+""";
	    """,
	    'comment':"""
//At least one scenario needs to be in the quantile
"""},
	4:{
	    'opl':"""
 quantile_biodiversity>=1/"""+str(no_realization)+""";
	    """,
	    'comment':"""
//At least one scenario needs to be in the quantile
"""}
    },
    'extra_parameters':"""

    """,
#    'data_file':"LARGE_DATASET/Large_dataset.dat AB.dat"
#    'data_file':"SMALL_DATASET/SMALL_dataset.dat AB.dat"
    'data_file':"MEDIUM_DATASETs/dataset_300.dat AB.dat"

    }

### Print the Achievement scalarizing problem into an string
The bounds of the variables and the reference point are to be added later

In [ ]:
def print_problem(optimized_objective_names,solution_file):
    model_ach = problem['data']
    model_ach+="{string} objective_names = {"
    sign = ""
    for obj_no in problem['objectives'].keys():
        model_ach+=sign+"\""+problem['objectives'][obj_no]['name']+"\""
        sign=","
    model_ach+="};\n"
    model_ach+="{string} optimized_objective_names = {"
    sign = ""
    for obj_name in optimized_objective_names:
        model_ach+=sign+"\""+obj_name+"\""
        sign=","
    model_ach+="};\n"
    model_ach+="float ref_point[objective_names] = ["
    sign = ""
    for obj_no in problem['objectives'].keys():
        model_ach+=sign+"%("+problem['objectives'][obj_no]['name']+"_refpoint)s"
        sign = ","
    model_ach+="];\n"
    model_ach+="float minimized[objective_names]= ["
    sign = ""
    for obj_no in problem['objectives'].keys():
        model_ach+=sign+problem['objectives'][obj_no]['minimized']
        sign = ","
    model_ach+="];\n"
    model_ach+="float normalization_factor[objective_names] = ["
    sign = ""
    for obj_no in problem['objectives'].keys():
        model_ach+=sign+"(%("+problem['objectives'][obj_no]['name']+"_ub)s-%("+problem['objectives'][obj_no]['name']+"_lb)s)"
        sign = ","
    model_ach+="];\n"
    model_ach+="float lb[objective_names] = ["
    sign = ""
    for obj_no in problem['objectives'].keys():
        model_ach+=sign+"%("+problem['objectives'][obj_no]['name']+"_lb)s"
        sign = ","
    model_ach+="];\n"
    for var_no in problem['variables'].keys():
        model_ach+=problem['variables'][var_no]['opl']
        model_ach+="\n"
    model_ach+="dvar float objective_value[objective_names];\n"
    model_ach+= "maximize \n min(obj_name in optimized_objective_names) (objective_value[obj_name]-ref_point[obj_name])*1000/normalization_factor[obj_name]+"
    model_ach+=str(rho)+"*sum(obj_name in objective_names) objective_value[obj_name]*1000/normalization_factor[obj_name];\n"
    model_ach+="""
    subject to
    {
    """
    for obj_no in problem['objectives'].keys():
        model_ach+=problem['objectives'][obj_no]['comment']
        model_ach+="objective_value[\""+problem['objectives'][obj_no]['name']+"\"]=="+problem['objectives'][obj_no]['opl']+";\n"
    for constr_no in problem['constraints'].keys():
        model_ach+=problem['constraints'][constr_no]['comment']
        model_ach+=problem['constraints'][constr_no]['opl']
    model_ach+="\n}"



    model_ach+="\n\nexecute\n{\n"
    #model_ach+="writeln(\"==OBJECTIVES==\")\n"
    model_ach+="var ofile = new IloOplOutputFile(\"objectives.csv\");\n"
    sign = ""
    for obj_no in problem['objectives'].keys():
    #    model_ach+= "writeln(\""+problem['objectives'][obj_no]['name']+"=\"+objective_value[\""+problem['objectives'][obj_no]['name']+"\"]);\n"
        model_ach+= "ofile.write(\""+sign+"\"+objective_value[\""+problem['objectives'][obj_no]['name']+"\"]);\n"
        sign = ","
    model_ach+="ofile.close();"

    model_ach+="var ofile = new IloOplOutputFile(\"variables.csv\");\n"
    #model_ach+="writeln(\"==VARIABLES==\")\n"
    for var_no in problem['variables'].keys():
    #    model_ach+="writeln(\""+problem['variables'][var_no]['name'] + "= \"+"+problem['variables'][var_no]['name'] +");\n"
        model_ach+="ofile.writeln(\""+problem['variables'][var_no]['name'] + "= \"+"+problem['variables'][var_no]['name'] +");\n"
    model_ach+="ofile.close();"
    model_ach+="}"


    model_ach+="\n\nmain\n{\n"
    model_ach+="var model = thisOplModel; \n" 
    model_ach+="model.generate();\n"
    model_ach+="cplex.readMIPStarts(\""+solution_file+"\");\n"
    model_ach+="cplex.solve();\n"
    model_ach+=problem['extra_parameters'] +"\n"
#    model_ach+="cplex.writeMIPStarts(\""+solution_file+".mst\", 0, 1000);\n"
    model_ach+="thisOplModel.postProcess();\n"
    model_ach+="}"
    return model_ach


<h3> Estimate nadir and idea values

The following code uses the *print_problem* function to estimate the nadir and ideal values of the objectives

In [ ]:
rho = 0
import csv
bound_candidates = []


#ub_dummy = [1 for obj_no in problem['objectives'].keys()]
ub_dummy = [5246600.76009152,520.496,4828656.0397274,519.783,1,1]
lb_dummy = [0 for obj_no in problem['objectives'].keys()]

lb_dict = {problem['objectives'][obj_no]['name']+"_lb":lb_dummy[obj_no] for obj_no in problem['objectives'].keys()}
ub_dict = {problem['objectives'][obj_no]['name']+"_ub":ub_dummy[obj_no] for obj_no in problem['objectives'].keys()}
reference = [0 for obj_no2 in problem['objectives'].keys()]
reference_dict = {problem['objectives'][obj_no]['name']+"_refpoint":reference[obj_no] for obj_no in problem['objectives'].keys()}
From_GUI_dict = dict(reference_dict.items()+lb_dict.items()+ub_dict.items())



for obj_no in problem['objectives'].keys():

    model_ach_with_reference = print_problem([problem['objectives'][obj_no]['name']],"solution"+str(obj_no)) % From_GUI_dict
    f = open('problem_ach_with_reference_'+str(obj_no)+'.mod','w')
    f.write(model_ach_with_reference)
    f.close()
    !oplrun -v -conflict problem_ach_with_reference_{str(obj_no)}.mod {problem['data_file']}
    with open('objectives.csv','rb') as csvfile:
        obj_reader = csv.reader(csvfile,delimiter = ",")
        for row in  obj_reader:
            bound_candidates.append([float(value) for value in row])
    !mv variables.csv variables{str(obj_no)}.csv
    !mv objectives.csv objectives{str(obj_no)}.csv
bound_candidates = zip(*bound_candidates)
ub = [max(bounds) for bounds in bound_candidates]
lb = [min(bounds) for bounds in bound_candidates]

print lb, ub

#Make sure that the reference point is never the same as the bounds (basically using the ideal, but slightly easier)
lb = [a-rho for a in lb]
ub = [a+rho for a in ub]

### Calculating the neutral comporomise solution

The following script calculates the neutral compromise solution for the problem

In [ ]:
rho = 0.0001


lb_dict = {problem['objectives'][obj_no]['name']+"_lb":lb[obj_no] for obj_no in problem['objectives'].keys()}
ub_dict = {problem['objectives'][obj_no]['name']+"_ub":ub[obj_no] for obj_no in problem['objectives'].keys()}

reference = [i/2 for i in ub_dummy]
reference_dict = {problem['objectives'][obj_no]['name']+"_refpoint":reference[obj_no] for obj_no in problem['objectives'].keys()}
From_GUI_dict = dict(reference_dict.items()+lb_dict.items()+ub_dict.items())
model_ach_with_reference = print_problem([problem['objectives'][obj_no]['name'] for obj_no in problem['objectives'].keys()],"solution2") % From_GUI_dict
f = open('problem_ach_with_reference_reference1.mod','w')
f.write(model_ach_with_reference)
f.close()

In [ ]:
!oplrun -v -conflict problem_ach_with_reference_reference1.mod {problem['data_file']}

## Solving the achievement scalarizing problem for a given reference point

In [ ]:
rho = 0.0001

lb_dict = {problem['objectives'][obj_no]['name']+"_lb":lb[obj_no] for obj_no in problem['objectives'].keys()}
ub_dict = {problem['objectives'][obj_no]['name']+"_ub":ub[obj_no] for obj_no in problem['objectives'].keys()}

#This is the reference to be changed w.r.t. the preferences of the decision maker
reference = [2500000, 504, 2600000, 505, 0.7, 0.85]

reference_dict = {problem['objectives'][obj_no]['name']+"_refpoint":reference[obj_no] for obj_no in problem['objectives'].keys()}
From_GUI_dict = dict(reference_dict.items()+lb_dict.items()+ub_dict.items())
model_ach_with_reference = print_problem([problem['objectives'][obj_no]['name'] for obj_no in problem['objectives'].keys()],"solution2") % From_GUI_dict
f = open('problem_ach_with_reference_reference1.mod','w')
f.write(model_ach_with_reference)
f.close()

In [ ]:
!oplrun -v -conflict problem_ach_with_reference_reference1.mod {problem['data_file']}